<a href="https://colab.research.google.com/github/Yujin-nKim/dev-archive/blob/%F0%9F%A5%AF_yujin/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠️ 1. 준비
- 런타임 > 런타임 유형 변경 > GPU 선택

### 📦 1.1 라이브러리 설치

In [1]:
!pip install transformers konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 29.5 MB/s eta 0:00:00


### 📁 1.2 데이터 셋 다운로드
- [웹크롤링 뉴스 데이터 csv (header = 카테고리, 뉴스 제목, 뉴스 내용, 뉴스 링크)](https://github.com/IceVanillaLatte-CanTech/dev-archive/blob/main/dataset/dataset_news_data_detailed.csv)
- 위 데이터셋을 다운받고 코랩 파일 탭에 업로드하기

In [3]:
import pandas as pd

# CSV 파일 읽기
news_data = pd.read_csv("dataset_news_data_detailed.csv")

news_data.head()

,카테고리,뉴스 제목,뉴스 내용,뉴스 링크
0,금융,"“우리 동창회 총무, 돈 관리 잘 하고 있나”…이 통장 만들면 서로 편하다는데","저비용으로 효과 커 효자상품\n신한銀, 3년 만에 재출시 나서\n저축은행도 상품 출...",https://n.news.naver.com/mnews/article/009/000...
1,금융,[이코노미 플러스]지난해 불법사금융 피해신고 급증…채권추심 4년만에 5배로,지난해 불법사금융 피해신고 급증…채권추심 4년만에 5배로\n사진=연합뉴스\n사진=연...,https://n.news.naver.com/mnews/article/087/000...
2,금융,[이코노미 플러스] 금융당국 저축은행 여러곳 '취약' 추가통보…적기시정조치 가능성,금융당국 저축은행 여러곳 '취약' 추가통보…적기시정조치 가능성\n자산건전성 악화 잇...,https://n.news.naver.com/mnews/article/087/000...
3,금융,[이코노미 플러스]공매도 통합 가이드라인 마련…내부통제기준 법인별 차등화,금융당국이 오는 3월 말 공매도 재개를 앞두고 규제 체계를 명확히 하는 '통합 가이...,https://n.news.naver.com/mnews/article/087/000...
4,금융,[이코노미 플러스] “상호금융 주담대 부실 가능성…리스크 관리 강화해야”,"보험연, 은행 보험사 대비 채무불이행자 비율↑\n◇보험연구원 제공\n\n\n상호금융...",https://n.news.naver.com/mnews/article/087/000...


In [4]:
# 데이터 크기 출력
print(f"데이터 크기 | (뉴스 개수, 열 개수) : {news_data.shape}")

데이터 크기 | (뉴스 개수, 열 개수) : (7837, 4)


# 🧹 2. 데이터 전처리

### 🩹 2.1 결측값 확인 및 제거

In [5]:
# 결측값 제거 전 개수 확인
print("결측값 개수:")
print(news_data.isnull().sum())

결측값 개수:
카테고리      0
뉴스 제목    10
뉴스 내용    55
뉴스 링크     0
dtype: int64


In [6]:
title_missing = news_data[(news_data["뉴스 제목"].isna()) & (news_data["뉴스 내용"].notna())]
print(f"🔹 뉴스 제목만 없는 경우: {len(title_missing)}개")

🔹 뉴스 제목만 없는 경우: 0개


In [7]:
content_missing = news_data[(news_data["뉴스 제목"].notna()) & (news_data["뉴스 내용"].isna())]
print(f"🔹 뉴스 내용만 없는 경우: {len(content_missing)}개")

🔹 뉴스 내용만 없는 경우: 45개


In [8]:
both_missing = news_data[(news_data["뉴스 제목"].isna()) & (news_data["뉴스 내용"].isna())]
print(f"🔹 뉴스 제목과 뉴스 내용 둘 다 없는 경우: {len(both_missing)}개")

🔹 뉴스 제목과 뉴스 내용 둘 다 없는 경우: 10개


뉴스 데이터 셋의 유형은 다음과 같다.
1. 뉴스 제목만 있는 경우 (본문에 이미지만 포함된 경우)
2. 뉴스 제목과 뉴스 내용이 모두 있는 경우

데이터셋을 활용할 때, 뉴스 제목 또는 뉴스 내용 중 하나라도 존재하는 경우 유효한 데이터로 간주하며, 두 항목 모두 없는 경우에는 결측값으로 판단해 제거한다.

In [9]:
# 뉴스 제목과 뉴스 내용이 모두 결측값인 행 제거
news_data = news_data.dropna(subset=["뉴스 제목", "뉴스 내용"], how="all")

In [10]:
# 결측값 제거 후 결측값 개수 확인
print("결측값 개수:")
print(news_data.isnull().sum())

결측값 개수:
카테고리      0
뉴스 제목     0
뉴스 내용    45
뉴스 링크     0
dtype: int64


In [11]:
# NaN 값을 빈 문자열로 채움
news_data["뉴스 내용"] = news_data["뉴스 내용"].fillna("")

### 🧽 2.2 텍스트 정제

<details>
  <summary>뉴스 제목 + 뉴스 내용 예시 (정제 전)</summary>
“우리 동창회 총무, 돈 관리 잘 하고 있나”…이 통장 만들면 서로 편하다는데	저비용으로 효과 커 효자상품\n신한銀, 3년 만에 재출시 나서\n저축은행도 상품 출시 채비\n\n\n[사진 = 연합뉴스]\n금융권이 신규 고객 유치의 수단으로 여러 사람이 함께 계좌를 관리할 수 있는 ‘모임통장’을 잇달아 내놓고 있다. 모임통장은 출시·관리에 큰 비용이 들지 않지만 많은 자금을 조달할 수 있어 효자 상품이다. 차별화를 위해 단체적금, 영수증 첨부 기능 등을 내놓는 곳도 등장했다.\n\n20일 금융권에 따르면 신한은행은 다음달 ‘SOL모임통장’을 출시한다. 신한은행은 2011년 모임통장 전용 앱 ‘김총무’를 출시했지만 저조한 이용률 등을 이유로 2022년 6월 서비스를 중단했다가 3년 만에 재출시한다.\n\n시중은행 모임통장 상품 중 최초로 적금통장 기능을 갖췄다. 모임 멤버들이 특정 목표를 위해 금액과 기간을 설정하고 다 같이 돈을 모을 수 있도록 했다. 신한은행 고객이 아니더라도 모바일 웹을 통해 모임통장에 참여할 수 있다. 과거 금융권에서 출시된 모임통장은 해당 금융사 이용자가 앱을 설치해야만 참여가 가능했다.\n\n거래내역 영수증 첨부 기능도 더해 통장에 들어오거나 나가는 돈의 액수와 사용처를 투명하게 관리할 수 있도록 했다. 참여 멤버들에게 모임통장의 거래 세부 내용을 속이는 방식으로 ‘곗돈 먹튀’가 발생하는 일을 막을 수 있다는 게 신한은행 측 설명이다. 모임통장에 모인 돈을 한동안 쓸 일이 없을 때 파킹통장에 넣어둬 이자를 더 받는 것도 가능하다.\n\nKB국민은행도 모임통장 서비스를 운영 중이다. 지난해 하반기 선보인 ‘KB모임금고’라는 파킹통장 서비스다. 모임통장 서비스에 가입한 고객이 여유자금을 보관할 수 있도록 하는 상품이며 최고금리는 연 2.0%다.\n\n우리은행은 지난해 11월 새 뱅킹 앱인 ‘뉴원뱅킹’ 출시와 함께 앱에 모임통장 기능을 신설했다. 거래 내역은 그대로 두고 모임장을 교체할 수 있으며, 하나의 모임에서 카드 여러 장을 발급받을 수 있도록 했다.\n\n하나은행에서도 ‘하나원큐’ 앱을 통해 모임통장 서비스 이용이 가능하다. 기존에 사용 중이던 통장에 모임 기능을 연결해 모임통장으로 쓸 수 있다. 납부일·미납사실 자동 알림 기능을 갖췄으며, 음식점이나 커피전문점 등에서 모임 전용 체크카드 이용 시 월 최대 1만원의 캐시백을 받을 수 있다. NH농협은행 또한 ‘올원뱅크’ 앱과 연계한 ‘NH모여라통장’을 운영 중이다.\n\n모임통장은 수시 입출금통장 형태가 대부분이다. 기본금리는 연 0.1% 수준이다. 카카오뱅크, 케이뱅크, 토스뱅크와 같은 인터넷전문은행도 다양한 상품을 내놓고 있다.\n\n저축은행도 조만간 모임통장 서비스에 뛰어들 예정이다. 저축은행중앙회는 모임통장 시스템 구축 절차를 밟고 있다.\n\n시중은행 관계자는 “은행은 모임통장으로 일반 예·적금 상품보다 많은 자금을 조달할 수 있다”면서 “모임에 참여하는 사람들이 은행의 다른 서비스를 이용하도록 유도할 수 있다는 것도 장점”이라고 설명했다.
  
</details>

#### 텍스트 정제 대상
1. 불필요한 개행 문자 (\n)
```
 효자상품\n신한銀, 3년 만에 재출시 나서\n
```

2. 기사와 직접 관련 없는 메타데이터
```
< 사진 제공 : 비알랩 >
<사진 제공=한국중소벤처기업유통원>
< ‘소호’ 실크벽지 56177-2 >
[데일리안 = 김소희 기자]
[대구=뉴시스]
기자
```

3. 한글 자음/모음 제거 (단독으로 존재하는 한글 자음이나 모음 제거)
```
 LNG 발전 용량이 2024년 2043ㄹGW(기가와트)에서
```

In [12]:
import re

# 텍스트 정제 함수 정의
def text_clean(text):
    # 불필요한 개행 문자 제거
    text = text.replace('\n', ' ')

    # 기사와 직접 관련 없는 메타데이터 제거
    text = re.sub(r'<[^>]*>', '', text)

    # 기자 이름 및 메타데이터 제거 (예: [서울=연합뉴스])
    text = re.sub(r'\[.*?\]|기자', '', text)

    # 한글 자음/모음 제거 (단독으로 존재하는 한글 자음이나 모음 제거)
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', text)

    return text

In [13]:
# 뉴스 제목과 뉴스 내용 정제
news_data["뉴스 제목 정제"] = news_data["뉴스 제목"].apply(text_clean)
news_data["뉴스 내용 정제"] = news_data["뉴스 내용"].apply(text_clean)

In [14]:
# 데이터 준비  : 뉴스 제목과 내용을 하나의 텍스트 데이터로 통합
news_data["통합 텍스트"] = news_data["뉴스 제목 정제"] + " " + news_data["뉴스 내용 정제"]

news_data["통합 텍스트"] = news_data["통합 텍스트"].fillna("").astype(str)

In [15]:
# 필요한 부분만 추출
news_data = news_data[["카테고리", "통합 텍스트"]]

In [16]:
news_data.head()

,카테고리,통합 텍스트
0,금융,"“우리 동창회 총무, 돈 관리 잘 하고 있나”…이 통장 만들면 서로 편하다는데 저비..."
1,금융,지난해 불법사금융 피해신고 급증…채권추심 4년만에 5배로 지난해 불법사금융 피해신고...
2,금융,금융당국 저축은행 여러곳 '취약' 추가통보…적기시정조치 가능성 금융당국 저축은행 ...
3,금융,공매도 통합 가이드라인 마련…내부통제기준 법인별 차등화 금융당국이 오는 3월 말 공...
4,금융,"“상호금융 주담대 부실 가능성…리스크 관리 강화해야” 보험연, 은행 보험사 대비 ..."


In [17]:
# 카테고리 문자열과 숫자를 매핑
category_mapping = {
    "금융": 0,
    "증권": 1,
    "산업/재계": 2,
    "중기/벤처" : 3,
    "부동산" : 4,
    "글로벌 경제" : 5,
    "생활경제" : 6,
    "경제일반" : 7
}

news_data["카테고리"] = news_data["카테고리"].map(category_mapping)

In [18]:
news_data["카테고리"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

# 📚 3. 모델 학습

### 🧪 3.1 테스트 데이터 분할

In [19]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(news_data, test_size=0.2, random_state=42)

train_texts = train_df["통합 텍스트"].astype(str).tolist()
train_labels = train_df["카테고리"].tolist()

test_texts = test_df["통합 텍스트"].astype(str).tolist()
test_labels = test_df["카테고리"].tolist()

### 🔑 3.2 KoBERT 토크나이저 불러오기 & 토큰화

In [20]:
from transformers import AutoTokenizer

# KoBERT 토크나이저 로드
KoBERT_tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

# 텍스트 데이터 토큰화
train_encodings = KoBERT_tokenizer(train_texts, truncation=True, padding=True)
test_encodings = KoBERT_tokenizer(test_texts, truncation=True, padding=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

### 🚚 3.3 DataLoader 구성

In [21]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)
batch_size = 16  # 배치 사이즈
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### 🧠 3.4 KoBERT 모델 불러오기

In [22]:
from transformers import AutoModelForSequenceClassification

# KoBERT 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    'monologg/kobert',
    num_labels=8
)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 🏋️ 3.5 모델 훈련

In [23]:
import torch
from tqdm.auto import tqdm # 진행률 표시

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용 설정

num_epochs = 10
learning_rate = 2e-5  # 0.00002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
model.to(device)  # 모델을 GPU로 이동

for epoch in range(num_epochs):
    model.train()  # 훈련 모드 지정
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)

        loss = criterion(outputs.logits, labels)  # 손실 계산
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        # 정확도 계산
        predictions = torch.argmax(outputs.logits, dim=1)  # 가장 높은 확률의 클래스 선택
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    average_loss = total_loss / len(train_loader)
    accuracy = correct / total * 100  # 정확도를 백분율로 변환
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f} - Accuracy: {accuracy:.2f}%")

  0%|          | 0/392 [00:00<?, ?it/s]

Epoch 1/10 - Average Loss: 1.3001 - Accuracy: 58.01%


  0%|          | 0/392 [00:00<?, ?it/s]

Epoch 2/10 - Average Loss: 0.8698 - Accuracy: 72.40%


  0%|          | 0/392 [00:00<?, ?it/s]

...

Epoch 8/10 - Average Loss: 0.3941 - Accuracy: 85.85%


  0%|          | 0/392 [00:00<?, ?it/s]

Epoch 9/10 - Average Loss: 0.3586 - Accuracy: 86.81%


  0%|          | 0/392 [00:00<?, ?it/s]

Epoch 10/10 - Average Loss: 0.3272 - Accuracy: 88.05%


In [24]:
# 모델 저장
model_save_path = "model.pth"  # 저장 경로
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to model.pth


### ✅ 3.6 모델 테스트

In [25]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        correct_predictions += torch.sum(predicted_labels == labels).item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7241


# 🔍 4. 추론

In [30]:
# 입력 문장 (정답 4)
input_text = """ 건설경기 진짜 어렵다…지난해 건설사 폐업신고 641건, 19년 만에 최고
지난해 종합건설기업의 폐업 신고 건수가 641건으로 19년 만에 최고치를 기록했다.

건설업 취업자 수도 지속적으로 감소하는 등 당분간 건설경기 불황이 이어질 것이란 전망이다.

1일 한국건설산업연구원에 따르면 지난해 종합건설기업의 폐업 신고는 전년보다 60건(10.3%) 늘어난 641건으로 집계됐다. 이는 조사가 시작된 2005년(629건) 이후 가장 많은 수치다.

폐업 신고는 2021년 305건에서 2022년 362건으로 증가한 후, 2023년(581건)에 이어 지난해까지 3년 연속 증가했다.

반면 지난해 새로 등록한 종합건설기업은 가장 최근 집계인 지난해 10월까지 1만9242곳으로, 전년 말(1만9516곳)보다 274곳(1.4%) 줄었다.

부문별로 건축업(1만493곳)이 전년 말보다 225곳(2.1%) 감소했다. 토건(3044)은 38곳(1.2%), 토목(5222곳)은 21곳(0.4%) 줄었다.

폐업한 기업은 늘고 새로 등록한 기업이 줄어든 것은 건설 경기가 좋지 않다는 뜻이다. 지속적인 공사비 급등, 미분양 증가, 입주 물량 감소 등으로 건설사들의 수익성은 악화하고 있다.

이런 가운데 건설 투자는 줄고 건설업 취업자 수도 감소하는 추세다. 건설 투자를 뜻하는 건설기성액은 지난해 11월 기준 13조9000억원으로 지난해 같은 기간보다 10.7% 줄었다.

지난해 6월 15조7000억원이었던 건설기성액은 7월 13조2000억원으로 꺾이기 시작해 11월까지 5개월 연속 감소했다.

지난해 11월 건설 공사비 지수는 130.3으로 전년 같은 기간보다 3.9% 상승하면서 건설 물가 부담은 커졌다.

같은 기간 건설업 취업자는 209만명으로 4.4% 감소했다. 취업자는 전월 대비로는 지난 5월부터 7개월 연속 줄었다.

2023년 하반기부터 지난해 초까지 건축공사 마감 공사에 많은 인력이 투입됐는데, 지난해 5월부터 이들 공사가 완공되면서 건축 현장도 많이 감소했기 때문으로 분석됐다.

건산연 관계자는 “폐업 업체 수는 증가하고 등록업체 수는 위축되는 가운데 건설업 취업자 수가 감소하는 등 전형적인 건설경기 침체 상황에 있는 것으로 판단된다”고 말했다.
"""

In [31]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# GPU/CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# KoBERT 모델 및 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", num_labels=8)
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

# 저장된 모델 가중치 불러오기
model.load_state_dict(torch.load("model.pth", map_location=device))
model.to(device)
model.eval()  # 평가 모드 설정

print("Model and tokenizer loaded successfully!")

# 토크나이저를 사용하여 입력 변환
input_encoding = tokenizer.encode_plus(
    input_text,
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# 입력 텐서를 GPU/CPU로 이동
input_ids = input_encoding["input_ids"].to(device)
attention_mask = input_encoding["attention_mask"].to(device)

# 모델 추론
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    _, predicted_labels = torch.max(outputs.logits, dim=1)  # 가장 높은 확률을 가진 클래스 선택

# 예측된 클래스 출력
predicted_labels = predicted_labels.item()
print(f"Predicted label: {predicted_labels}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


<ipython-input-31-999535ee72a3>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth", map_location=device))


Model and tokenizer loaded successfully!
Predicted label: 4
